- add isholiday
- remove os version
- remove less influential feats (user_typeid, category_id)
- try categorical fe

In [1]:
!pip install jpholiday 

In [2]:
import os
import jpholiday
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn import preprocessing
from tqdm._tqdm_notebook import tqdm_notebook
from sklearn.metrics import average_precision_score
from sklearn.model_selection import StratifiedKFold
tqdm_notebook.pandas(desc="progress: ")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  import sys
/opt/conda/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
DIR = "../input/20201115at/"
train = pd.read_csv(DIR+"train.csv")
test = pd.read_csv(DIR+"test.csv")
campaign = pd.read_csv(DIR+"campaign.csv")

#advertiser_video = pd.read_csv(DIR+"advertiser_video.csv")
#map_game_feed_native_video_assets = pd.read_csv(DIR+'map_game_feed_native_video_assets.csv')

#advertiser_converted_video = pd.read_csv(DIR+"advertiser_converted_video.csv")

# preprocess

In [4]:
# with campaign
train = train.merge(campaign, left_on="campaign_id", right_on = "id", how="left").drop(["mst_advertiser_id", "id"], axis=1)
test = test.merge(campaign, left_on="campaign_id", right_on = "id", how="left").drop(["mst_advertiser_id", "id"], axis=1)

# with map_gae_feed_native_video_assets
#train = train.merge(map_game_feed_native_video_assets, left_on=["game_feed_id"], right_on = ["mst_game_feed_id"], how="left").drop(["mst_game_feed_id"], axis=1)
#test = test.merge(map_game_feed_native_video_assets, left_on=["game_feed_id"], right_on = ["mst_game_feed_id"], how="left").drop(["mst_game_feed_id"], axis=1)

# with advertiser_video horizontal
#horizontal = advertiser_video.copy()
#horizontal.columns = [f"horizontal_{c}" if c not in ["id", "mst_advertiser_id"] else c for c in horizontal.columns]
#train = pd.merge(train, horizontal[["id", "mst_advertiser_id", "horizontal_duration"]], left_on=["horizontal_mst_advertiser_video_id", "advertiser_id"], 
#               right_on=["id", "mst_advertiser_id"], how="left").drop(columns=["id", "mst_advertiser_id"])
#test = pd.merge(test, horizontal[["id", "mst_advertiser_id", "horizontal_duration"]], left_on=["horizontal_mst_advertiser_video_id", "advertiser_id"], 
#               right_on=["id", "mst_advertiser_id"], how="left").drop(columns=["id", "mst_advertiser_id"])

In [5]:
train["imp_at"] = pd.to_datetime(train["imp_at"])
test["imp_at"] = pd.to_datetime(test["imp_at"])

In [6]:
# drop because distribution is different
train.drop(['os_version'], axis=1, inplace=True)
test.drop(['os_version'], axis=1, inplace=True)

In [7]:
# noise features?
train.drop(['user_type_id', "category_id",  "video_template_id", "country_code",  "pos", "game_feed_asset_type_id"], axis=1, inplace=True)
test.drop(['user_type_id', "category_id",  "video_template_id", "country_code", "pos", "game_feed_asset_type_id"], axis=1, inplace=True)

In [8]:
# drop because the exist only in train
train.drop(['click_or_view_at', 'cv_at','last_cv_interval', 'last_paid_interval'], axis=1, inplace=True)

In [9]:
#object_feats = [i for i in train.columns if "object" == str(train[i].dtype)]
#train[object_feats] = train[object_feats].fillna("-999")
#test[object_feats] = test[object_feats].fillna("-999")

#for i in object_feats:
#    print(i)
#    lbl = preprocessing.LabelEncoder()
#    lbl.fit(list(pd.concat([train[i], test[i]])))
#    train[i] = lbl.transform(list(train[i]))
#    test[i] = lbl.transform(list(test[i]))

In [10]:
object_feats = [i for i in train.columns if "object" == str(train[i].dtype)]

def get_non_overlapping(train: pd.DataFrame, test: pd.DataFrame, column: str):
    only_in_train = set(train[column].unique()) - set(test[column].unique())
    only_in_test = set(test[column].unique()) - set(train[column].unique())
    non_overlapping = only_in_train.union(only_in_test)
    return non_overlapping


def category2num(train: pd.DataFrame, test: pd.DataFrame, columns: list):
    train_ = train.copy()
    test_ = test.copy()
    for column in columns:
        non_overlapping = get_non_overlapping(train, test, column)
        try:
            if train[column].dtype == np.dtype("O"):
                # dtypeがobjectなら欠損は'missing' クラスにする
                train_[column] = train[column].fillna("missing")
                test_[column] = test[column].fillna("missing")
                train_[column] = train_[column].map(lambda x: x if x not in non_overlapping else "other")
                test_[column] = test_[column].map(lambda x: x if x not in non_overlapping else "other")
            else:
                # dtypeがint/floatなら欠損は'-1'とする
                train_[column] = train[column].fillna(-1)
                test_[column] = test[column].fillna(-1)
                train_[column] = train_[column].map(lambda x: x if x not in non_overlapping else -2)
                test_[column] = test_[column].map(lambda x: x if x not in non_overlapping else -2)

            le = preprocessing.LabelEncoder()
            concatenated = pd.concat([train_, test_], axis=0).reset_index(drop=True)
            le.fit(concatenated[column])
            train_[column] = le.transform(train_[column])
            test_[column] = le.transform(test_[column])
        except Exception:
            import pdb
            pdb.set_trace()
    return train_, test_

train, test = category2num(train, test, object_feats)

# feature engineering

In [11]:
train["type"] = "train"
test["type"] = "test"
all_df = pd.concat([train, test])
all_df = all_df.sort_values(["uid", "imp_at"]).reset_index(drop=True)

In [12]:
all_df["count"] = 1
all_df["count"] = all_df.groupby("uid")["count"].cumsum()

In [13]:
%%time
dates = all_df.imp_at.dt.date
dts_map = {}
for dt in dates:
    dts_map[dt] = jpholiday.is_holiday(dt)

# 祝日
all_df["holiday"] = all_df.imp_at.dt.date.map(dts_map)
all_df["holiday"] = all_df["holiday"].astype(int)

# 祝休日
#all_df["is_off_day"] = all_df["dayofweek"].map(lambda x: 1 if x in [5, 6] else 0) + all_df["holiday"]

CPU times: user 5min 57s, sys: 165 ms, total: 5min 58s
Wall time: 5min 58s


In [14]:
#all_df["prev_app"] = all_df.groupby("uid")["app_id"].progress_apply(lambda x: x.shift(1))

#all_df["uni_item"] = all_df.groupby("uid")["item_id"].progress_apply(lambda x: 1- x.duplicated())
#all_df["uni_item"] = all_df.groupby("uid")["uni_item"].progress_apply(lambda x: x.cumsum())

#all_df["login_freq_cumsum"] = all_df.groupby("uid")["login_frequency"].cumsum()

#AGGS = [
#    ('login_frequency', 2, ['uid'], ['mean']),
#]

#for on, lag, by, hows in AGGS:
#    agg = all_df.groupby(by)[on].apply(lambda x: (
#        x.shift(1).rolling(window=lag, min_periods=1).agg(hows)
#    ))
#    agg = agg.rename(columns={
#        how: f'{how}_of_last_{lag}_{on}_by_' + '_and_'.join(by)
#        for how in hows
#    })

#    all_df = all_df.join(agg)

In [15]:
train = all_df[all_df.type=="train"].drop("type", axis=1).reset_index(drop=True)
test = all_df[all_df.type=="test"].drop("type", axis=1).reset_index(drop=True)

In [16]:
train = train.sort_values("imp_at").reset_index(drop=True)
test = test.sort_values("imp_at").reset_index(drop=True)

# separation for validation

In [17]:
train.drop(['request_id'], axis=1, inplace=True)
test.drop(['request_id'], axis=1, inplace=True)

In [18]:
mask = train.imp_at > "2020-05-21"
valid_df = train.loc[mask].copy()
train_df = train.loc[~mask].copy()

In [19]:
train = train.drop(["imp_at"], axis=1)
test = test.drop(["imp_at"], axis=1)

valid_df = valid_df.drop(["imp_at"], axis=1)
train_df = train_df.drop(["imp_at"], axis=1)

In [20]:
test = test.drop("target", axis=1)

In [21]:
train.shape, test.shape 

((1997595, 26), (390095, 25))

In [22]:
train.head()

,target,adnw_id,adspot_id,adspot_video_format_id,advertiser_id,app_id,auction_type_id,campaign_id,first_login_interval,frequency,...,last_login_interval,login_frequency,max_login_interval,media_app_id,os,uid,mst_advertiser_order_id,mst_user_type_id,count,holiday
0,0.0,4,1111,2.0,14,5653,1.0,256,1490644.0,0,...,11320.0,10.0,2607.0,4780,1,108518,25,35,1,0
1,0.0,28,1081,2.0,86,3127,2.0,56,790412.0,2,...,12589.0,2.0,13540.0,3645,2,108518,115,37,2,0
2,0.0,42,1123,3.0,4,4947,1.0,76,1506105.0,0,...,10254.0,9.0,33725.0,121,1,108518,17,35,3,0
3,0.0,8,1113,2.0,15,4725,1.0,47,184283.0,0,...,39618.0,4.0,7377.0,2466,1,108518,112,38,4,0
4,0.0,22,1061,NaN,95,67,4.0,214,1999495.0,2,...,35652.0,5.0,164759.0,867,1,108518,38,35,5,0


# stratified modelling

In [23]:
object_feats = [i for i in object_feats if i in train.columns] + ["holiday"]

In [24]:
# stratified k fold
n_folds = 5
def pr_auc_metric(y_predicted, y_true):
    return 'pr_auc', average_precision_score(y_true.get_label(), y_predicted), True

def modelling(seed):
    print("seed:", seed)
    lgbm_params = {'objective': 'binary', 'metric': 'None', 'boosting_type': 'gbdt', 
               'tree_learner': 'serial', 'learning_rate': 0.1, "num_leaves": 10, 'random_seed':seed,'max_depth': 5}
    
    y = train.target.copy()
    X = train.drop(['target'],axis=1).copy()
    
    skf=StratifiedKFold(n_splits = n_folds, shuffle=True, random_state=0)
    
    valid = np.zeros([X.shape[0]])
    features_list = [i for i in X.columns]
    pred_value = np.zeros(test.shape[0])
    feature_importance_df = pd.DataFrame(features_list, columns=["Feature"])
    
    for i , (train_index, val_index) in enumerate(skf.split(X, y)):
        print("fold: ", i)
        
        X_train = X.iloc[train_index,:]
        y_train = y.iloc[train_index]

        X_valid = X.iloc[val_index,:]
        y_valid = y.iloc[val_index]
        
        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)
        
        model = lgb.train(lgbm_params, lgb_train,valid_sets=[lgb_train, lgb_eval],
           num_boost_round=10000,early_stopping_rounds=20,verbose_eval = 10,
                          feval=pr_auc_metric, categorical_feature = sorted(object_feats))
        
        valid_predict = model.predict(X_valid, num_iteration = model.best_iteration)
        valid[val_index] = valid_predict
        pred_value += model.predict(test, num_iteration = model.best_iteration)
        feature_importance_df["Fold_"+str(i+1)] = model.feature_importance()
    
    feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
    feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
    feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]
    
    return pred_value, valid, feature_importance_df

validation = np.zeros([train.shape[0]])
pred_value = np.zeros([test.shape[0]])
seeds = [0]
for seed_ in seeds:
    test_pred, oof_predict, feature_importance_df = modelling(seed_)
    validation += oof_predict / len(seeds)
    pred_value += test_pred / len(seeds)

score = average_precision_score(train.target, validation)
print(score)

seed: 0
fold:  0


/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['advertiser_id', 'app_id', 'campaign_id', 'game_feed_id', 'game_template_id', 'holiday', 'media_app_id', 'mst_advertiser_order_id', 'uid']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 20 rounds
[10]	training's pr_auc: 0.205545	valid_1's pr_auc: 0.201252
[20]	training's pr_auc: 0.214135	valid_1's pr_auc: 0.209232
[30]	training's pr_auc: 0.222541	valid_1's pr_auc: 0.215468
[40]	training's pr_auc: 0.22995	valid_1's pr_auc: 0.218977
[50]	training's pr_auc: 0.235604	valid_1's pr_auc: 0.222348
[60]	training's pr_auc: 0.241034	valid_1's pr_auc: 0.224813
[70]	training's pr_auc: 0.245711	valid_1's pr_auc: 0.22673
[80]	training's pr_auc: 0.248728	valid_1's pr_auc: 0.228045
[90]	training's pr_auc: 0.252181	valid_1's pr_auc: 0.229833
[100]	training's pr_auc: 0.254975	valid_1's pr_auc: 0.230303
[110]	training's pr_auc: 0.257047	valid_1's pr_auc: 0.231412
[120]	training's pr_auc: 0.259443	valid_1's pr_auc: 0.231897
[130]	training's pr_auc: 0.261849	valid_1's pr_auc: 0.23235
[140]	training's pr_auc: 0.263885	valid_1's pr_auc: 0.232811
[150]	training's pr_auc: 0.26532	valid_1's pr_auc: 0.232976
[160]	training's pr_auc: 0.267926	val

In [25]:
feature_importance_df.sort_values("Average", ascending=False)

,Feature,Fold_1,Fold_2,Fold_3,Fold_4,Fold_5,Average,Std,Cv
4,app_id,491,418,546,421,383,451.8,58.710817,0.129949
6,campaign_id,338,323,380,330,285,331.2,30.432877,0.091887
18,media_app_id,275,190,288,230,190,234.6,41.190290,0.175577
20,uid,239,187,255,226,166,214.6,33.109515,0.154285
10,game_feed_id,167,131,176,132,109,143.0,24.843510,0.173731
3,advertiser_id,118,115,117,108,109,113.4,4.127953,0.036402
21,mst_advertiser_order_id,91,74,123,81,82,90.2,17.267310,0.191434
23,count,93,79,90,78,70,82.0,8.414274,0.102613
7,first_login_interval,80,66,87,76,56,73.0,10.881176,0.149057
15,last_login_interval,77,65,80,74,56,70.4,8.777243,0.124677


# ts modelling

In [26]:
n_folds = 5
def pr_auc_metric(y_predicted, y_true):
    return 'pr_auc', average_precision_score(y_true.get_label(), y_predicted), True

def modelling_ts(seed):
    print("seed:", seed)
    lgbm_params = {'objective': 'binary', 'metric': 'None', 'boosting_type': 'gbdt', 
               'tree_learner': 'serial', 'learning_rate': 0.1, "num_leaves": 10, 'random_seed':seed,'max_depth': 5}
    y_train = train_df.target.copy()
    X_train = train_df.drop(['target'],axis=1).copy()
    y_valid = valid_df.target.copy()
    X_valid = valid_df.drop(['target'],axis=1).copy()
    
    valid_lgb = pd.DataFrame(np.zeros([X_train.shape[0]]))
    real = np.array([])
    features_list = [i for i in X_train.columns]
    feature_importance_df = pd.DataFrame(features_list, columns=["Feature"])
        
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)
        
    model = lgb.train(lgbm_params, lgb_train,valid_sets=[lgb_train, lgb_eval],
       num_boost_round=10000,early_stopping_rounds=20,verbose_eval = 10, feval=pr_auc_metric, categorical_feature = sorted(object_feats))
    
    valid_predict = model.predict(X_valid, num_iteration = model.best_iteration)
    feature_importance_df["Importance"] = model.feature_importance()
        
    score = average_precision_score(y_valid, valid_predict)
    print("average precision score = {}".format(score))
    pred_value = model.predict(test, num_iteration = model.best_iteration)
    
    return pred_value, valid_predict, feature_importance_df

#validation_ts = np.zeros([valid_df.shape[0]])
#pred_value_ts = np.zeros([test.shape[0]])
#seeds = [0]
#for seed_ in seeds:
#    test_pred, oof_predict, feature_importance_df = modelling_ts(seed_)
#    validation_ts += oof_predict / len(seeds)
#    pred_value_ts += test_pred / len(seeds)

#score = average_precision_score(valid_df.target, validation_ts)
#print(score)

In [27]:
#feature_importance_df.sort_values("Importance", ascending=False)

# submission

In [28]:
lgb_sub = pd.read_csv(DIR + "atmaCup7__sample_submission.csv")
lgb_sub["target"] = pred_value
lgb_sub.to_csv("atmacup7_lgb"+str(score)[:-10]+".csv", index = False)